# Detecção de Reclamações de Agência

Este notebook implementa o pipeline completo:
1. Preprocessamento
2. Tokenização + POS tagging
3. Matching de sequências + sinônimos
4. Cálculo de features
5. Treino de modelos offline (TF-IDF, Word2Vec, LDA)
6. Score final e threshold
7. Avaliação de performance

In [32]:
# !pip install spacy gensim scikit-learn unidecode
# !python -m spacy

In [33]:
import re
import string
import numpy as np
import pandas as pd
from unidecode import unidecode
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from gensim.models import Word2Vec
from gensim.corpora import Dictionary
from gensim.models.ldamodel import LdaModel

# Carrega modelo SpaCy offline pt-br
# spacy.cli.download('pt_core_news_sm')
# nlp = spacy.load("/caminho/até/pt_core_news_sm") ## SALVO NO GITHUB
nlp = spacy.load('pt_core_news_sm')

# Função de pré-processamento
def preprocessar(texto: str) -> str:
    texto = texto.lower()
    texto = unidecode(texto)
    texto = re.sub(r"\d+", "", texto)
    texto = re.sub(f"[{re.escape(string.punctuation)}]", " ", texto)
    texto = re.sub(r"\s+", " ", texto).strip()
    return texto

# Função de tokenização + POS tagging
def tokenizar_com_pos(texto: str):
    doc = nlp(texto)
    return [token for token in doc]

In [34]:
TERMOS = [
    {'seq': ['gerente', 'atendimento', 'ruim'], 'peso': 0.9},
    {'seq': ['manipulado', 'gerente', 'agencia'], 'peso': 0.9},
    {'seq': ['falta', 'de', 'contato'], 'peso': 0.8},
    {'seq': ['tentativas', 'contato', 'frustrad'], 'peso': 0.8},
    {'seq': ['tempo', 'de', 'espera'], 'peso': 0.6},
    {'seq': ['caixa', 'eletronico', 'erro'], 'peso': 0.6},
    {'seq': ['demora', 'retorno'], 'peso': 0.7},
    {'seq': ['funcionario', 'inexperiente'], 'peso': 0.6},

    # --- PESO MÁXIMO (1.0): Indicações inequívocas de falha de contato ---
    {'seq': ['ninguem', 'atende'], 'peso': 1.0},
    {'seq': ['nao', 'tive', 'retorno'], 'peso': 1.0},
    {'seq': ['falta', 'de', 'comunicacao'], 'peso': 1.0},

    # --- PESO ALTO (0.9): Termos muito fortes e diretos ---
    {'seq': ['falta', 'de', 'retorno'], 'peso': 0.9},
    {'seq': ['falta', 'de', 'contato'], 'peso': 0.9},
    {'seq': ['sem', 'retorno'], 'peso': 0.9},
    {'seq': ['sem', 'resposta'], 'peso': 0.9},
    {'seq': ['nao', 'atende', 'telefone'], 'peso': 0.9},
    {'seq': ['nao', 'responde'], 'peso': 0.9},
    {'seq': ['email', 'ignorado'], 'peso': 0.9},
    {'seq': ['me', 'ignora'], 'peso': 0.9},
    {'seq': ['comunicacao', 'inexistente'], 'peso': 0.9},

    # --- PESO MÉDIO-ALTO (0.8): Termos fortes, mas com leve espaço para ambiguidade ---
    {'seq': ['demora', 'retorno'], 'peso': 0.8},
    {'seq': ['prazo', 'estourou'], 'peso': 0.8},
    {'seq': ['impossivel', 'falar'], 'peso': 0.8},
    {'seq': ['impossivel', 'contato'], 'peso': 0.8},
    {'seq': ['tentativas', 'frustradas'], 'peso': 0.8}, # Mantido e melhorado
    {'seq': ['prometeu', 'retorno'], 'peso': 0.8},
    {'seq': ['ficou', 'de', 'retornar'], 'peso': 0.8},

    # --- PESO MÉDIO (0.7): Termos relacionados e que reforçam o contexto ---
    {'seq': ['aguardando', 'retorno'], 'peso': 0.7},
    {'seq': ['aguardando', 'resposta'], 'peso': 0.7},
    {'seq': ['nao', 'me', 'avisou'], 'peso': 0.7},
    {'seq': ['nenhuma', 'satisfacao'], 'peso': 0.7},
    {'seq': ['chamada', 'caiu'], 'peso': 0.7}, # Frequentemente mencionado em tentativas frustradas

    # --- PESO MAIS BAIXO (0.6): Termos de contexto geral de negligência ---
    {'seq': ['total', 'descaso'], 'peso': 0.6}, # Termo mais geral, mas altamente correlacionado
    {'seq': ['tempo', 'de', 'espera'], 'peso': 0.6} # Relevante para espera telefônica
]

SINONIMOS = {
    'atendimento':   ['suporte', 'assistencia', 'apoio'],
    'ruim':          ['péssimo', 'descaso', 'insatisfatorio'],
    'frustrad':     ['sem exito', 'sem sucesso', 'vao'],
    'internet':      ['email', 'online'],
    'cancelado':     ['anulado', 'descontinuado'],
    'espera':        ['fila', 'aguardo'],
    'erro':          ['falha', 'bug', 'pane'],
    'demora':        ['atraso', 'lentidao', 'lento', 'demorado', 'espera'],
    'solucionou':    ['resolveu', 'corrigiu'],
    'inexperiente':  ['novato', 'iniciante'],

    # Termos centrais de comunicação
    'contato':       ['comunicacao', 'ligacao', 'interacao', 'chamada', 'recado', 'mensagem', 'email', 'telefone'],
    'retorno':       ['resposta', 'posicao', 'satisfacao', 'feedback', 'retornar', 'responder'],
    'atender':       ['responder', 'atende', 'responde', 'falou', 'atendeu', 'falar'],
    'ligar':         ['telefonar', 'chamar', 'discar', 'retornar a ligacao', 'ligou', 'ligaria'],
    'avisar':        ['informar', 'comunicar', 'notificar', 'avisado', 'informado', 'avisou'],
    
    # Termos que indicam ausência ou falha
    'falta':         ['sem', 'ausencia', 'inexistencia', 'nenhum', 'nenhuma', 'nada', 'impossivel'],
    'ignorar':       ['ignorado', 'ignora', 'ignoram', 'ignorar'],
    'cair':          ['caiu', 'desconectou', 'desligou', 'cai a ligacao'],
    'esperar':       ['aguardar', 'aguardando', 'esperando', 'na espera', 'no aguardo'],
    
    # Termos de frustração e descaso
    'descaso':       ['absurdo', 'negligencia', 'desrespeito', 'indiferenca', 'falta de atencao', 'inacreditavel'],
    'urgente':       ['urgencia', 'emergencia', 'imediato', 'rapido', 'prioridade'],
    
    # Pessoas e Prazos
    'gerente':       ['responsavel', 'atendente', 'funcionario', 'gerencia', 'banco'],
    'prometer':      ['prometeu', 'garantiu', 'afirmou', 'ficou de'],
    'prazo':         ['data', 'limite', 'tempo', 'data limite'],
    'estourar':      ['estourou', 'venceu', 'expirou', 'passou do prazo'],
    
    # Problemas e solicitações
    'resolver':      ['solucionar', 'ajudar', 'verificar', 'dar andamento'],
    'problema':      ['pendencia', 'solicitacao', 'duvida', 'protocolo', 'questão'],
}

from itertools import product

def expandir_sequencia(seq, sinonimos):
    pools = []
    for token in seq:
        pool = [token]
        if token in sinonimos:
            pool.extend(sinonimos[token])
        pools.append(pool)
    return [list(p) for p in product(*pools)]

# Expansão completa
EXPANDIDAS = []
for termo in TERMOS:
    seqs = expandir_sequencia(termo['seq'], SINONIMOS)
    for s in seqs:
        EXPANDIDAS.append({'seq': s, 'peso': termo['peso']})

In [35]:
# Definição das Features
def contagem_ponderada(tokens):
    palavras = [t.text for t in tokens]
    soma = 0.0
    for item in EXPANDIDAS:
        seq = item['seq']
        peso = item['peso']
    for i in range(len(palavras) - len(seq) + 1):
        if palavras[i:i+len(seq)] == seq:
            soma += peso
    return soma

def distancia_minima(tokens):
    palavras = [t.text for t in tokens]
    pos_ag = [i for i,w in enumerate(palavras) if w == 'agencia']
    pos_neg = []
    for item in EXPANDIDAS:
        tok = item['seq'][0]
        pos_neg += [i for i,w in enumerate(palavras) if w == tok]
    if not pos_ag or not pos_neg:
        return len(palavras)
    return min(abs(pa - pn) for pa in pos_ag for pn in pos_neg)

In [36]:
# Treino de Modelos Offline
def treinar_tfidf(corpus):
   vec = TfidfVectorizer()
   X = vec.fit_transform(corpus)
   prototipo = np.asarray(X.mean(axis=0)).ravel()
   return vec, prototipo

def sim_tfidf(texto, vec, prototipo):
   v = vec.transform([texto])
   proto = prototipo.reshape(1, -1)
   return cosine_similarity(v, proto)[0,0]

def treinar_w2v(corpus_tokens, tamanho=100):
   sentencas = [[t.text for t in tokens] for tokens in corpus_tokens]
   modelo = Word2Vec(sentences=sentencas, vector_size=tamanho, window=5, min_count=1)
   return modelo

def sim_w2v(tokens, modelo):
   vetores = [modelo.wv[t.text] for t in tokens if t.text in modelo.wv]
   if not vetores:
      return 0.0
   media = np.mean(vetores, axis=0).reshape(1, -1)
   ref = modelo.wv['agencia'].reshape(1, -1) if 'agencia' in modelo.wv else media
   return cosine_similarity(media, ref)[0,0]

def treinar_lda(corpus_tokens, num_temas=5):
   textos = [[t.text for t in tokens] for tokens in corpus_tokens]
   dicionario = Dictionary(textos)
   bow = [dicionario.doc2bow(doc) for doc in textos]
   lda = LdaModel(corpus=bow, id2word=dicionario, num_topics=num_temas)
   return lda, dicionario

def sim_lda(tokens, lda, dicionario, id_tema=0):
   bow = dicionario.doc2bow([t.text for t in tokens])
   temas = lda.get_document_topics(bow)
   for tid, prob in temas:
      if tid == id_tema:
            return prob
   return 0.0

In [37]:
import pandas as pd
df = pd.read_excel('reclamacoes.xlsx')
df.head(2)

texto  label
0  Ao contrário do que pensei, o atendimento na a...    NaN
1  O caixa da agência não aceitou meu depósito e ...    NaN

In [38]:
# Prepara e treina modelos
corpus_txts = [preprocessar(t) for t in df['texto']]
corpus_toks = [tokenizar_com_pos(t) for t in corpus_txts]
vec, prototipo = treinar_tfidf(corpus_txts)
w2v_mod = treinar_w2v(corpus_toks)
lda_mod, lda_dic = treinar_lda(corpus_toks)

# Função de cálculo de score
def calcular_score(texto):
    pre = preprocessar(texto)
    toks = tokenizar_com_pos(pre)
    f1 = contagem_ponderada(toks)
    f2 = distancia_minima(toks)
    f3 = sim_tfidf(pre, vec, prototipo)
    f4 = sim_w2v(toks, w2v_mod)
    f5 = sim_lda(toks, lda_mod, lda_dic)
    # combinação linear com pesos fixos
    return (1.0 * f1 + -0.1 * f2 + 1.0 * f3 + 0.5 * f4 + 0.5 * f5)

In [39]:
# Aplica e exibe scores
df['score'] = df['texto'].apply(calcular_score)
df[['texto','score']].sort_values(by='score', ascending=False)

texto     score
0   Ao contrário do que pensei, o atendimento na a...  0.816320
39  Deixei meu contato para ser informado sobre a ...  0.610206
35  É um absurdo a completa falta de retorno da ag...  0.589426
40  Estou aguardando o retorno sobre a análise do ...  0.394610
43  Total descaso. A agência não faz o mínimo, que...  0.378674
30  A agência simplesmente não atende as ligações....  0.373940
16  Falta de comunicação da agência, ninguém retor...  0.358502
36  O canal de comunicação com a agência não funci...  0.342634
2   O gerente da agência foi extremamente rude e n...  0.336986
29  Estou há mais de uma semana tentando contato t...  0.329929
23  Há dias tento contato com a agência e ninguém ...  0.326552
42  É inacreditável ter que implorar por um retorn...  0.314291
1   O caixa da agência não aceitou meu depósito e ...  0.295933
44  Já tentei todos os canais possíveis, mas a agê...  0.266653
34  Meu gerente visualiza minhas mensagens no apli...  0.258431
38  Abri um protocolo na agência e o prazo para re...  0.255138
11  Agência sem internet, não consigo emitir compr...  0.226391
3   Experiência frustrada: agência sem gerente dis...  0.222958
19  A fila de espera na agência estava muito longa...  0.211471
12  Não recebi o extrato mensal via app e a agênci...  0.194227
33  Enviei três e-mails para o endereço da agência...  0.182342
4   Fui à agência e o atendimento do gerente foi p...  0.142049
6   Erro no caixa eletrônico da agência: prendeu m...  0.119513
5   Central de atendimento automática mal funciona...  0.096047
9   Tive fraude na conta e a agência demorou dias ...  0.042269
37  Fui presencialmente na agência para resolver u...  0.022718
41  A comunicação com a minha agência é simplesmen... -0.018287
31  Passei a manhã inteira tentando falar na agênc... -0.083571
17  Estou há dias tentando contato com minha agênc... -0.103630
8   Meu pedido de empréstimo na agência está parad... -0.117731
10  O responsável pela minha conta na agência nunc... -0.151834
14  A agência cancelou meu horário agendado sem av... -0.205888
13  Visitei a agência hoje e fui atendido por um f... -0.378702
15  Tentei visitar a agência para desbloquear meu ... -0.645714
28  Ligo para a agência todos os dias e o telefone... -0.660706
7   Cartão de crédito bloqueado indevidamente, som... -0.699839
18  O problema foi resolvido, mas o atendimento fo... -0.785173
24  O aplicativo só funciona em wifi, bloqueando o... -0.970525
21  O aplicativo de internet banking apresentou er... -1.018120
22  O sistema online caiu justo no dia que eu prec... -1.050501
20  Meu cartão chegou com defeito e a central de s... -1.062928
27  Site do banco fora do ar no fim de semana, dei... -1.276661
26  Essa empresa é horrível, o aplicativo é um lix... -1.299158
25  Tive um problema com meu cartão de crédito e o... -1.780136
32  A atendente prometeu que o responsável pela mi... -1.848394

## Similaridade

In [42]:
def encontrar_texto_mais_similar(texto_alvo, lista_textos, sinonimos_dict):
    """
    Encontra o texto em uma lista que é mais similar a um texto alvo,
    considerando um dicionário de sinônimos.

    Args:
        texto_alvo (str): O texto que você quer comparar.
        lista_textos (list): Uma lista de textos (ex: uma coluna de DataFrame).
        sinonimos_dict (dict): Dicionário de sinônimos.

    Returns:
        tuple: Uma tupla contendo (texto_mais_similar, score_maximo).
               Retorna (None, -1) se a lista estiver vazia.
    """
    
    # --- PASSO 1: Inverter o dicionário de sinônimos para busca O(1) ---
    mapa_sinonimos = {}
    for base, lista_sinonimos in sinonimos_dict.items():
        for sinonimo in lista_sinonimos:
            mapa_sinonimos[sinonimo] = base

    def normalizar_e_vetorizar(texto):
        """
        Função auxiliar para normalizar texto com sinônimos e gerar um vetor de documento.
        Esta versão é mais robusta e funciona com todos os modelos do spaCy.
        """
        doc = nlp(texto.lower())
        
        tokens_normalizados = []
        for token in doc:
            palavra_base = mapa_sinonimos.get(token.lemma_, token.lemma_)
            tokens_normalizados.append(palavra_base)
            
        # Recria um documento spaCy com as palavras normalizadas e usa seu vetor
        # O método .vector é a forma segura de obter a representação vetorial de um texto.
        doc_normalizado = nlp(" ".join(tokens_normalizados))
        return doc_normalizado.vector

    # --- PASSO 2: Vetorizar o texto alvo ---
    vetor_alvo = normalizar_e_vetorizar(texto_alvo).reshape(1, -1)
    
    if not lista_textos:
        return None, -1
        
    # --- PASSO 3: Vetorizar todos os textos da lista de comparação ---
    vetores_lista = [normalizar_e_vetorizar(texto) for texto in lista_textos]
    
    # --- PASSO 4: Calcular a similaridade de cosseno ---
    similaridades = cosine_similarity(vetor_alvo, vetores_lista)
    
    # --- PASSO 5: Encontrar o índice do mais similar e retornar ---
    indice_max = np.argmax(similaridades)
    score_maximo = similaridades[0, indice_max]
    texto_mais_similar = lista_textos[indice_max]
    
    return texto_mais_similar, score_maximo

In [ ]:
# Suponha que esta seja a coluna de textos do seu DataFrame
base_de_reclamacoes = df['texto'].tolist()

# O novo texto que queremos classificar ou encontrar um similar
nova_reclamacao = "Mandei um email e o banco não me respondeu."

# Encontrar o texto mais similar na nossa base
texto_similar, score = encontrar_texto_mais_similar(
    texto_alvo=nova_reclamacao,
    lista_textos=base_de_reclamacoes,
    sinonimos_dict=SINONIMOS
)

print(f"Novo texto: '{nova_reclamacao}'")
print("-" * 30)
print(f"Texto mais similar encontrado na base: '{texto_similar}'")
print(f"Score de Similaridade de Cosseno: {score:.4f}")

Novo texto: 'Mandei um email e o banco não me respondeu.'
------------------------------
Texto mais similar encontrado na base: 'Meu gerente visualiza minhas mensagens no aplicativo mas não responde. A agência precisa me dar um retorno sobre a minha solicitação.'
Score de Similaridade de Cosseno: 0.8632
